# Boston housing

#### Attribute Information:
#### 1)  CRIM    - 자치시별 1인당 범죄율
#### 2)  ZN      - 25000평방피트를 초과하는 거주지역의 비율
#### 3)  INDUS   - 비소매상업지역이 점유하고 있는 토지의 비율
#### 4)  CHAS    - 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1,아니면 0)
#### 5)  NOX     - 10ppm당 농축 일산화질소
#### 6)  RM      - 주택 1가구당 평균 방의개수
#### 7)  AGE     - 1940년 이전에 건축된 소유주택의 비율
#### 8)  DIS     - 5개의 보스턴 직업센터까지의 접근성 지수
#### 9)  RAD     - 방사형 도로까지의 접근성 지수
#### 10) TAX     - 10,000달러 당 재산세율
#### 11) PTRATIO - 자치시(town)별 학생/교사 비율
#### 12) B       - 1000(Bk - 0.63)^2 , bk는 자치시의 흑인의 비율을 뜻함
#### 13) LSTAT   - 모집단의 하위계층의 비율(%)
#### 14) MEDV    - 본인소유의 주택가격(중앙값) (단위:$1.000)

# 0. 데이터 로드 

In [1]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.datasets import load_boston

load_boston = load_boston()

dfX = pd.DataFrame(load_boston.data, columns=load_boston.feature_names)
dfy = pd.DataFrame(load_boston.target, columns=["MEDV"])
boston = pd.concat([dfX, dfy], axis=1)

print(boston.shape)
print(boston.head())
# x_train, x_test, y_train, y_test = train_test_split(dataset['data'], dataset['target'])

(506, 14)
      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  


# 1. 데이터 탐색 및 전처리

#### 데이터 탐색에서은 아래 프로세스로 데이터를 탐색 후 전처리 한다

#### 1. 데이터의 분포 확인 ( 기초 통계량 - MIN, MAX, QUANTILE, 결측치) 
#### 2. SKWED 데이터 셋 추출 및 리스케일링
#### 3. 이상치 데이터 감지 및 변환 작업
####    - 이상치 데이터는 이상치의 데이터가 굉장히 적을경우 (1% 미만)는 그냥 삭제 하거나
####    - 다른값으로 대치(Imputation)
####    - 치우쳐진 경우 스케일링작업(log,제곱근)
####    - Min,max,Z-score등으로 정규화시킨다.


##### (1) 데이터 탐색작업단계에서 ('ZN', 'B', 'CRIM') 세 변수가 skewed 한 분포를 보임과 동시에 이상치 데이터가 감지되는 데이터로
#####     파악 할 수 있음
##### (2) (1)의 변수는 아래 Feature선정시 해당 데이터가 종속변수와의 연관성이 있는지를 파악한 후, 제외할지 스케일링하여 input데이터로 사용할지
#####     결정 한다.
##### (3) 종속변수인 'MEDV'변수도 마찬가지로 이상치 데이터가 감지되었으며 총 506개의 데이터 중  40개(7.91%) 정도
#####     종속변수의 이상치데이터가 너무 많을시 모델링의 결과가 왜곡될 수 있으며, 40개의 데이터를 모두 제거하기엔 데이터의 수가 줄어들고
#####     과적합 현상이 생길 수 있음 MAX/MIN 값의 데이터만 절사 하는 수준으로 이상치 데이터를 제거 후 인풋데이터정제

In [9]:
# 데이터 기초 통계량 , 타입, 결측치 확인 
# 결측치 없음
# 데이터 타입은 모두 수치형 데이터
# CHAS 데이터는 더미변수로 히스토그램,박스플랏을 보는 EDA에서는 제외
print('===================================================================')
print('데이터 기초 통계량  \n \n\n', boston.describe())
print('\n 데이터  \n',boston.info())
# print('\n 결측값 확인 \n',boston.isnull().sum())
print('===================================================================')

# 데이터 분포 확인
# CRIM , ZN, B 변수의 경우 Skewed한 분포를 보이고 있으며
# CHAS 의 경우 바이너리 데이터로 대다수의 데이터가 0으로 포진되어 있다.

def EDA_distributions(df):    
    # except Binary data(chas)
    for feature in df.columns:
        print(feature)
        if feature == 'CHAS':
            print(Counter(df[feature]))
            print(' ')
        else :
            plt.figure(figsize=(15,6))
            plt.subplot(1, 2, 1)
            fig = sns.boxplot(y=df[feature])
            fig.set_title('')
            fig.set_ylabel(feature)
            
            #histplot + kdeplot= distplot
            plt.subplot(1, 2, 2)
            fig = sns.distplot(df[feature])
            fig.set_xlabel(feature)
            plt.show()    
        
        
# 이상치 탐색(iQR)
def detect_outliers(df):
    outlier_cols = []
    perc_list = []
    upper_limit = []
    lower_limit = []
    for col, data in df.items():
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        iqr = q3 - q1
        scale = 1.5
        lower = q1 - scale * iqr
        upper = q3 + scale * iqr
        v_col = data[(data < lower) | (data > upper)]
        perc =( np.shape(v_col)[0] / np.shape(boston)[0] )* 100.0
        print("Column %s outliers [ %s / %s ] = %.2f%%" % (col,np.shape(v_col)[0],np.shape(df)[0], perc))
        outlier_indices = np.shape(v_col)[0]
        if outlier_indices == 0:
            print('%s 변수에 탐지된 이상치는 없습니다.\n'%col)
        else:
            print('%s 변수에 탐지된 이상치 데이터의 개수  : %s\n'%(col,outlier_indices))
            outlier_cols.append(col)
            perc_list.append(perc)
            lower_limit.append(lower)
            upper_limit.append(upper)
    result = pd.DataFrame({'outlier_list' : outlier_cols,
                           'perc' : perc_list,
                           'upper_limit':upper_limit,
                           'lower_limit':lower_limit})
    
    return result.sort_values(by=['perc'],ascending = False)


def check_skewed(df):
    skew_df =  df.drop(['MEDV'], axis=1).skew()
    sked_df = abs(skew_df).sort_values(ascending=False)
    return sked_df[sked_df >= 2]

# Data Scaling
def log_scaling(df):
    input_df = df
    sked_list = ['CRIM', 'ZN','B']
    for col in sked_list:
        print(col)
        print('before : ',df[col].skew())
#         after_boxcox = stats.boxcox(input_df[col]+1)[0]
        aa = np.log(input_df[col])
        input_df[col] = pd.Series(aa)
        print('after : ',input_df[col].skew())
#         plt.subplot(1, 2, 2)
        fig = sns.distplot(input_df[col])
        fig.set_xlabel(col)
        plt.show() 

def  std_scaling(df):
    X = df.drop(['MEDV'], axis=1)
    y = df['MEDV']

    scaler = StandardScaler()
    cancer_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    return cancer_scale
        
#  데이터 분포 체크
EDA_distributions(boston)        
#  이상치 점검 및 이상치가 있는 데이터 감지
outlier_df = detect_outliers(boston.iloc[:,~boston.columns.isin(['CHAS'])])
print('[ outlier_list ]\n\n%s'%outlier_df)
#  SKED 데이터셋 추출 ( 왜도의 기준은 2이상으로 정의함 )
sked_list = check_skewed(boston).index.to_list()
print('=============================================================================================')
print('\noutlier 감지 리스트 : ', outlier_df['outlier_list'].to_list())
print('\nSKED한 데이터 리스트 : ', sked_list)
print('Skewed하고 이상치가 있는 데이터 : ',set(sked_list)&set(outlier_df['outlier_list'].to_list()))

데이터 기초 통계량  
 

              CRIM          ZN       INDUS        CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677083   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         RAD         TAX     PTRATIO           B  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534  356.674032   
std     28

### 종속변수 이상치 제거 

In [3]:
input_df = boston[(boston['MEDV'] > min(boston['MEDV']))&(boston['MEDV'] < max(boston['MEDV']))]

# 2. Feature 의 선정

### Feature selection에는 총 3가지 방법론을 적용하여 3가지 방법론의 교집합을 독립변수로 셀렉함.

### 1. 상관관계분석
#### --> 상관관계 분석시에는 상관계수의 절대값이 0.3이상인 약한 상관관계이상의 변수들만 셀렉한 후 
#### --> 다중공선성을 고려하여 각 독립변수간의 상관계수가 0.9이상인 데이터는 종속변수와의 상관계수가 높은 변수만 남기고 제거한다.
### 2. Feature Importance
#### --> Tree기반의 Feature Importance를 기준으로 시각화하면 importances rank가 0.01이상은 변수들만 셀렉
#### --> 해당 방법론 적용시 위의 아웃라이어&SKEWED 분포를 가진 변수들은 제외됨
### 3. 추정모델에 따른 feature의 weight를 기준으로 자동으로 selection해주는 selectfrommodel
#### --> threshold(임계값)을 변수 선택에 사용하여 중요도가 크거나 같은 변수는 셀렉하고 중요도가 낮은 변수는 제외시키는 로직

In [10]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso

def correlation_matrix(df):
    ## 상관관계가 0.3 이상인 강한 상관관계를 보이는 변수만 선택
    corr_matrix = boston.corr()
    top_corr_feature = corr_matrix.index[abs(corr_matrix['MEDV']) >= 0.3].to_list()
    except_cols = set(boston) - set(top_corr_feature)
    
    rank_corr_feature = abs(corr_matrix.loc[abs(corr_matrix['MEDV']) >= 0.3,['MEDV','CRIM']])
    rank_corr_feature = rank_corr_feature.sort_values(by=['MEDV'],ascending=False)
    
    print('MEDV(Y)와 상관계수가 0.3 이하인 약한 상관관계를 보이는 변수 : %s'%except_cols )
    plt.figure(figsize=(13,10))
    sns.heatmap(boston[top_corr_feature].corr(), annot = True)
    plt.show()
    
    # 다중공선성 ( 변수 선택 및 제거 )
    # 독립변수간의 상관관계가 강한 경우 모델의 과적합 현상이 발생할 수 있음
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find features with correlation greater than 0.9
    feature = [column for column in upper.columns if any(upper[column] >= 0.9)]
    ss = corr_matrix.TAX[corr_matrix.TAX >= 0.9].index.tolist()
    print('다중공선성이 보이는 변수  : %s '%(ss))
    final_feature = rank_corr_feature.drop(index='RAD', columns='MEDV').index.to_list()
    return [col for col in final_feature if col != 'MEDV']
    
    
def selectfrommodel(df):
    X = df.drop(['MEDV'], axis=1)
    y = df['MEDV']
    input_cols = X.columns
    lgb_selector   = SelectFromModel(estimator=LGBMRegressor()).fit(X, y)
    xgb_selector   = SelectFromModel(estimator=XGBRegressor()).fit(X, y)
    ridge_selector = SelectFromModel(estimator=Ridge()).fit(X, y)
    lasso_selector = SelectFromModel(estimator=Lasso()).fit(X, y)
    
    feature_list = list(input_cols[lgb_selector.get_support()])+\
                   list(input_cols[xgb_selector.get_support()])+\
                   list(input_cols[ridge_selector.get_support()])+\
                   list(input_cols[lasso_selector.get_support()])
    
#     print('LightGBM : ',input_cols[lgb_selector.get_support()])
#     print('xgboost  : ',input_cols[xgb_selector.get_support()])
#     print('Ridge : '   ,input_cols[ridge_selector.get_support()])
#     print('Lasso : '   ,input_cols[lasso_selector.get_support()])
    
    return list(set(feature_list))


#Meta-transformer for selecting features based on importance weights.

def Feature_Importance(df):
    X = df.iloc[:,~df.columns.isin(['MEDV'])]
    y = df['MEDV']
    # RandomForest
    clf = RandomForestRegressor(random_state=42, max_depth=6)
    clf.fit(X, y)
    feature_importance = clf.feature_importances_

    # plot
    df_fi = pd.DataFrame({'columns':X.columns, 'importances':feature_importance})
    df_fi = df_fi[df_fi['importances'] > 0] # importance가 0이상인 것만 
    df_fi = df_fi.sort_values(by=['importances'], ascending=False)

    fig = plt.figure(figsize=(15,7))
    ax = sns.barplot(df_fi['columns'], df_fi['importances'])
    ax.set_xticklabels(df_fi['columns'], rotation=80, fontsize=13)
    plt.tight_layout()
    plt.show()
    return  df_fi[df_fi['importances'] >= 0.01]
    
corr_feature = correlation_matrix(input_df)
print('다중공선성 제외 상관관계 변수 : ',corr_feature)
sfm_list = selectfrommodel(input_df)
print('\nSelectfromModel 변수 : ',sfm_list )
df_fi = Feature_Importance(boston)
importance_feature = df_fi['columns'].to_list()
print('\nFeature Importance 변수 : ',importance_feature)

intersection_feature = list(set(corr_feature)&set(sfm_list)&set(importance_feature))+['MEDV']
print('\n교집합 변수 : ',intersection_feature)

# 3. 예측 모형 학습 및 성능평가

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,r2_score
from IPython.core.display import Image
# import numpy as np

# from sklearn import svm

# train,test 셋 분할
def split_train_test(df,size):
    X = df.drop(['MEDV'],axis=1)
    Y = df['MEDV']
    # random state = seed for shuffle
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=size, random_state=123)
    return X_train, X_test, Y_train, Y_test

def evaludation(y_true,y_pred):
    mse = mean_squared_error(y_true,y_pred)
    r2 = r2_score(y_true, y_pred)
    rmse = (np.sqrt(mse))
    return r2,rmse,mse

final_df = input_df[intersection_feature]

# select feature scaling if skew() > 0.3 
for col in final_df.columns:
    if np.abs(final_df[col].skew()) > 0.3:
        final_df[col] = np.log1p(final_df[col])
    else : 
        print('0.3이하변수 %s'%col)
#     plt.figure(figsize=(15,10))
#     plt.subplot(1, 2, 1)
#     fig = sns.distplot(input_df[col])
#     fig.set_xlabel(col)
#     plt.show()   

#     plt.subplot(2, 2, 2)
#     fig = sns.distplot(final_df[col])
#     fig.set_xlabel(col)
#     plt.show()    
ori_X_train, ori_X_test, ori_Y_train, ori_Y_test = split_train_test(boston,0.3)
X_train, X_test, Y_train, Y_test = split_train_test(final_df,0.3)

print('X columns : ',X_train.columns.to_list())

print('original X_train : %s /original X_test : %s : '%(len(ori_X_train),len(ori_X_test)))
print('original Y_train : %s /original  Y_test : %s : '%(len(ori_Y_train),len(ori_Y_test)))

print('X_train : %s / X_test : %s : '%(len(X_train),len(X_test)))
print('Y_train : %s / Y_test : %s : '%(len(Y_train),len(Y_test)))


def clf_grid(cat,X_train,X_test,Y_train,Y_test):
    clf_list    = []
    cat_list    = []
    feature_len = []
    feature     = []
    x_train_len = []
    x_test_len  = []
    y_train_len = []
    y_test_len  = []
    r2_list     = []
    mse_list    = []
    rmse_list   = []
    
    # 선형회귀
    # 트리모델
    # 부스팅 기법 (XGB,LGB)
    reg_classifiers = [
        LinearRegression(),
        RandomForestRegressor(),
        XGBRegressor(),
        LGBMRegressor()
        ]

    for item in reg_classifiers:
        clf = item
        clf.fit(np.array(X_train), np.array(Y_train))
        y_pred = clf.predict(np.array(X_test))
        y_true = np.array(Y_test)
        r2,rmse,mse = evaludation(y_true,y_pred)
        clf_list.append(str(clf).split('(')[0])
        cat_list.append(cat)
        feature_len.append(len(X_train.columns))
        feature.append(str(X_train.columns.to_list()))
        x_train_len.append(len(X_train))
        x_test_len.append(len(X_test))
        y_train_len.append(len(Y_train))
        y_test_len.append(len(Y_test))
        r2_list.append(r2)
        mse_list.append(mse)
        rmse_list.append(rmse)
    eval_df = pd.DataFrame(list(zip(clf_list, cat_list,feature_len,feature,x_train_len,x_test_len,y_train_len,y_test_len,r2_list,mse_list,rmse_list)), \
               columns =['clf','cat', 'feature_len','feature', 'x_train_len','x_test_len', 'y_train_len','y_test_len', 'r2','mse','rmse']) 
    return eval_df

before_eval_df =clf_grid('before feature selection',ori_X_train,ori_X_test,ori_Y_train,ori_Y_test) 
before_eval_df = before_eval_df.sort_values(by=['mse'])
after_eval_df = clf_grid('After feature selection',X_train,X_test,Y_train,Y_test)
after_eval_df = after_eval_df.sort_values(by=['mse'])

print('============ Feature 선정/scaling 전 성능평가 ==========')
print(before_eval_df[['clf','r2','mse','rmse']])
print('============ Feature 선정/scaling 후 성능평가 ==========')
print(after_eval_df[['clf','r2','mse','rmse']])

print('================== Best Estimator ==================')
print(after_eval_df.iloc[0])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.3이하변수 RM
X columns :  ['TAX', 'PTRATIO', 'RM', 'AGE', 'CRIM', 'LSTAT', 'NOX']
original X_train : 354 /original X_test : 152 : 
original Y_train : 354 /original  Y_test : 152 : 
X_train : 341 / X_test : 147 : 
Y_train : 341 / Y_test : 147 : 


## 4. 결과해석 ( Explainer)

#### (3) 의 Best Estimator를 기준으로 한다.
#### Explainer모델로 가장 많이사용하는 lime을 통해 예측값에 대한 변수의 영향도는 어떤지 파악한다.
#### - lime은 개별 관측치를 대상으로 설명하기에 1개의 관측치를 예시로 들어 볼 수 있으며,

#### 관측치의 변수중 어떤변수가 어떤 방향으로(양/음) 영향도를 갖게되었는지 파악할 수 있다




In [12]:
from lime.lime_tabular import LimeTabularExplainer

clf = LGBMRegressor()
clf.fit(np.array(X_train), np.array(Y_train))
y_pred = clf.predict(np.array(X_test))

explainer = LimeTabularExplainer(X_train.values,
mode='regression',feature_names=X_train.columns)


# asking for explanation for LIME model
i = 10
x_obs = X_test.iloc[[i],:]
y_obs = Y_test.iloc[i]
y_obs_pred = clf.predict(np.array(X_test.iloc[[i],:]))
print('x_관측치 : \n',x_obs)
print('\ny_관측치 : ',y_obs)
print('y_관측치 대한 예측값 : ',round(float(y_obs_pred),2))
explanation = explainer.explain_instance(x_obs.values[0],clf.predict)
print(explanation)

explanation.show_in_notebook(show_table=True, show_all=True)


print( 'R2',explanation.score)

